<a href="https://colab.research.google.com/github/CanKeles5/ObjectDetection/blob/main/Detectron2-VisDrone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

     |████████████████████████████████| 276kB 12.5MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44074 sha256=4d6722f4a39905184b6b3784a95ac66f96b225b715061afa1b2de577e3697368
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
1.7.1+cu101 False
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.7)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
import torch
assert torch.__version__.startswith("1.7")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
     |████████████████████████████████| 6.8MB 118.7MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 2.2MB 20.2MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.3.post20210227-cp37-none-any.whl size=59001 sha256=769ed2ba7b71f0532da41a06d5f9ee37e0ff3424acf13b4426c9a2d7a5ff9705
  Stored in directory: /root/.cache/pip/wheels/89/4b/64/716bf9925b2bb322cf85281e7d34d36542847c544a85182d05
  Created wheel for iopath: filename=iopath-0.1.4-cp37-none-any.whl size=15571 sha256=97193ea9d7eff1aae248c4010c5a101f79cd07b6d3f17d9cbbcf671f750063ec
  Stored in directory: /root/.cache/pip/wheels/54/3b/fb/25b60ef5989e3f5fd76fdc575c922e6e59066ddbd1b44756a9
Successfully built fvcore iopath
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.0.0
    Uninstallin

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



In [ ]:
!unzip /content/drive/MyDrive/VisDrone2019-DET-train.zip -d /content/

Görüntülenen çıkış son 5000 satıra kısaltıldı.
 extracting: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000140.jpg  
 extracting: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000141.jpg  
 extracting: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000142.jpg  
 extracting: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000143.jpg  
 extracting: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000144.jpg  
  inflating: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000145.jpg  
 extracting: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000146.jpg  
 extracting: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000147.jpg  
 extracting: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000148.jpg  
 extracting: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000149.jpg  
  inflating: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000150.jpg  
  inflating: /content/VisDrone2019-DET-train/images

In [ ]:
from detectron2.structures import BoxMode

In [ ]:
from google.colab.patches import cv2_imshow
#/content/VisDrone2019-DET-train/annotations
def get_visdrone_dicts(img_path = "/content/VisDrone2019-DET-train/images", annot_path = "/content/VisDrone2019-DET-train/annotations" ):
  dataset_dicts = []
  for path, subdirs, files in os.walk(img_path):
   for filename in files:
     record = {}
     
     
     img_p = os.path.join(path, filename)
     anot_p = os.path.join(annot_path, filename[:-4] + '.txt')

     h, w = cv2.imread(img_p).shape[:2]

     record["file_name"] = img_p
     record["image_id"] = filename
     record["height"] = h
     record["width"] = w

     objs = []

     with open(anot_p) as fp:
       line = fp.readline()
       while line:
         line = line.replace("\n","") 
         vals = line.split (",")
         id = int(vals[5])

         '''if id == 0 or id == 11:
           line = fp.readline()
           continue
        '''
        
         b_left, b_top, b_width, b_height = list(map(float,vals[:4]))
         b_right, b_bottom = b_left + b_width, b_top - b_height
         
         obj = {
            "bbox": [b_left, b_top, b_width, b_height],
            "bbox_mode": BoxMode.XYWH_ABS,
            "segmentation": [],
            "category_id": id,
         }
         objs.append(obj)
         line = fp.readline()
     record["annotations"] = objs
     dataset_dicts.append(record)
         #print(dataset_dicts)
  return dataset_dicts
     #print(img_p)
     #cv2_imshow(image)
     

ignored regions(0), pedestrian(1), 
                     people(2), bicycle(3), car(4), van(5), truck(6), tricycle(7), awning-tricycle(8), bus(9), motor(10), 
                     others(11)

In [ ]:
DatasetCatalog.register("datset9",  lambda d=_: get_visdrone_dicts("/content/VisDrone2019-DET-train/images", "/content/VisDrone2019-DET-train/annotations"))
MetadataCatalog.get("datset9").set(thing_classes =["Ignored", "Pedestrian", "People", "Bicycle", "Car", "Van", "Truck", "Tricycle", "awning-tricycle", "Bus", "Motor", "Others"])
MetadataCatalog.get("datset9").thing_colors = [(0,255,0), (255,0,0), (255,0,0), (0,0,255), (0,0,255), (0,0,255),(0,0,255),(0,0,255),(0,0,255),(0,0,255),(0,0,255),(0,255,0)]
#MetadataCatalog.get("datset6").thing_colors = [(102,255,102), (102,255,255), (102,102,255), (0,0,255), (255,0,0), (0,255,0),(140,50,160),(50,200,100),(160,70,200),(100,200,20),(100,200,20),(200,100,100)]

In [ ]:
balloon_metadata = MetadataCatalog.get("datset9")

In [ ]:
dataset_dicts = get_visdrone_dicts("/content/VisDrone2019-DET-train/images", "/content/VisDrone2019-DET-train/annotations")

In [ ]:
balloon_metadata

In [ ]:
#!pip install Pillow==4.0.0
#!pip install PIL
#!pip install image

In [ ]:
from detectron2.utils.visualizer import ColorMode
for d in dataset_dicts[:5]:
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=balloon_metadata, scale=1.5, instance_mode=ColorMode.SEGMENTATION)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
import json
from pathlib import Path
from PIL import Image as PILImage
import IPython
import numpy as np
from math import trunc
import base64
from io import BytesIO

In [ ]:
annot_path = "/content/drive/MyDrive/visdronecoco.json"
img_path = "/content/VisDrone2019-DET-train/images"

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("datset9",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 1
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 8
cfg.SOLVER.BASE_LR = 0.0002  # pick a good LR
cfg.SOLVER.MAX_ITER = 4000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 12  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

RuntimeError: ignored

In [ ]:
<# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
# !unzip /content/drive/MyDrive/VisDrone2019-VID-test-dev.zip -d /content/

In [ ]:
# !unzip /content/drive/MyDrive/VisDrone2019-DET-test-dev.zip -d /content/

In [ ]:
#DatasetCatalog.register("datset10",  lambda d=d: get_visdrone_dicts("/content/images", "/content/annotations"))
#MetadataCatalog.get("datset10").set(thing_classes =["Ignored", "Pedestrian", "People", "Bicycle", "Car", "Van", "Truck", "Tricycle", "awning-tricycle", "Bus", "Motor", "Others"])
#MetadataCatalog.get("datset10").thing_colors = [(0,255,0), (255,0,0), (255,0,0), (0,0,255), (0,0,255), (0,0,255),(0,0,255),(0,0,255),(0,0,255),(0,0,255),(0,0,255),(0,255,0)]

In [ ]:
#balloon_metadata = MetadataCatalog.get("datset10")

In [ ]:
#dataset_dicts = get_visdrone_dicts("/content/images", "/content/annotations")

In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
#cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "/content/drive/MyDrive/outputson visdrone/model_final.pth")  # path to the model we just trained
#cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.4   # set a custom testing threshold
#cfg.DATASETS.TEST = ( )
#predictor = DefaultPredictor(cfg)

In [ ]:
#import random
#from detectron2.utils.visualizer import ColorMode
#from detectron2.utils.visualizer import Visualizer

#for d in random.sample(dataset_dicts, 5):
#    img = cv2.imread(d["file_name"])
#    visualizer = Visualizer(img[:, :, ::-1], metadata=balloon_metadata, scale=1.5,instance_mode=ColorMode.SEGMENTATION)
#    vis = visualizer.draw_dataset_dict(d)
#    cv2_imshow(vis.get_image()[:, :, ::-1])

In [ ]:
#from detectron2.utils.visualizer import ColorMode

#for d in random.sample(dataset_dicts, 5):    
#    im = cv2.imread(d["file_name"])
#    outputs = predictor(im)
#    v = Visualizer(im[:, :, ::-1],
#                   metadata=balloon_metadata, 
#                   scale=1.5, 
#                   instance_mode=ColorMode.SEGMENTATION   # remove the colors of unsegmented pixels
#    )
#    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
#    cv2_imshow(v.get_image()[:, :, ::-1])